In [4]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [6]:
df = pd.read_csv("Updated_Car_Sales_Data.csv")
df

,Car Make,Car Model,Year,Mileage,Price,Fuel Type,Color,Transmission,Options/Features,Condition,Accident
0,Hyundai,Tucson,2010,52554,44143.820,Hybrid,Black,Automatic,Heated Seats,Used,No
1,Land Rover,Range Rover,2016,115056,25414.060,Diesel,Silver,Manual,GPS,Used,No
2,Honda,Accord,2022,18044,28262.872,Electric,Yellow,Manual,"Bluetooth, Leather Seats, Backup Camera, GPS",Like New,Yes
3,Kia,Soul,2011,79251,28415.848,Hybrid,Orange,Manual,"Backup Camera, GPS, Heated Seats, Leather Seats",New,Yes
4,Volkswagen,Golf,2022,40975,31509.792,Electric,Silver,Automatic,"Leather Seats, Remote Start, Sunroof, Heated S...",New,Yes
...,...,...,...,...,...,...,...,...,...,...,...
22995,Aston Martin,Vantage,2021,46146,164033.000,Hybrid,Red,Manual,Leather Seats,Like New,No
22996,Aston Martin,DBX,2017,45415,213867.000,Hybrid,White,Automatic,Sunroof,Like New,No
22997,Porsche,Cayenne,2015,32402,145626.000,Petrol,Black,Automatic,Navigation,New,Yes
22998,Lamborghini,Aventador,2020,27640,224705.000,Hybrid,White,Manual,Bluetooth,Used,Yes


In [7]:
df["Recession"] = df["Year"].apply(lambda x: "Recession" if x <= 2009 else "Non-Recession")
df

,Car Make,Car Model,Year,Mileage,Price,Fuel Type,Color,Transmission,Options/Features,Condition,Accident,Recession
0,Hyundai,Tucson,2010,52554,44143.820,Hybrid,Black,Automatic,Heated Seats,Used,No,Non-Recession
1,Land Rover,Range Rover,2016,115056,25414.060,Diesel,Silver,Manual,GPS,Used,No,Non-Recession
2,Honda,Accord,2022,18044,28262.872,Electric,Yellow,Manual,"Bluetooth, Leather Seats, Backup Camera, GPS",Like New,Yes,Non-Recession
3,Kia,Soul,2011,79251,28415.848,Hybrid,Orange,Manual,"Backup Camera, GPS, Heated Seats, Leather Seats",New,Yes,Non-Recession
4,Volkswagen,Golf,2022,40975,31509.792,Electric,Silver,Automatic,"Leather Seats, Remote Start, Sunroof, Heated S...",New,Yes,Non-Recession
...,...,...,...,...,...,...,...,...,...,...,...,...
22995,Aston Martin,Vantage,2021,46146,164033.000,Hybrid,Red,Manual,Leather Seats,Like New,No,Non-Recession
22996,Aston Martin,DBX,2017,45415,213867.000,Hybrid,White,Automatic,Sunroof,Like New,No,Non-Recession
22997,Porsche,Cayenne,2015,32402,145626.000,Petrol,Black,Automatic,Navigation,New,Yes,Non-Recession
22998,Lamborghini,Aventador,2020,27640,224705.000,Hybrid,White,Manual,Bluetooth,Used,Yes,Non-Recession


In [10]:
df["Vehicle_Type"] = df["Car Model"]
app = Dash(__name__)
app.title = "Automobile Sales Dashboard"
app

In [11]:
app.layout = html.Div([
    html.H1("XYZ Automotives Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Statistics"),
        dcc.Dropdown(
            id="stats-dropdown",
            options=[
                {"label": "Recession Period Statistics", "value": "recession"},
                {"label": "Yearly Statistics", "value": "yearly"}
            ],
            value="recession"
        )
    ]),

    html.Div([
        html.Label("Select Year"),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": str(y), "value": y} for y in sorted(df["Year"].unique())],
            value=sorted(df["Year"].unique())[0]
        )
    ]),

    html.Div(id="output-container", className="output-container")
])

In [13]:
@app.callback(
    Output("year-dropdown", "disabled"),
    Output("output-container", "children"),
    Input("stats-dropdown", "value"),
    Input("year-dropdown", "value")
)
def update_dashboard(stats, year):
    if stats == "recession":
        disabled = True

        rec_df = df[df["Recession"] == "Recession"]
        fig1 = px.pie(rec_df, names="Vehicle_Type", values="Price", title="Ad Spend by Vehicle Type (Recession)")

        fig2 = px.line(rec_df, x="Year", y="Price", title="Sales Trend in Recession")

        return disabled, [
            dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2)
        ]

    else:
        disabled = False

        year_df = df[df["Year"] == year]

        fig1 = px.bar(year_df, x="Vehicle_Type", y="Price", title=f"Vehicle Sales in {year}")

        fig2 = px.scatter(year_df, x="Mileage", y="Price", title=f"Advertising vs Sales in {year}")

        return disabled, [
            dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2)
        ]

if __name__ == "__main__":
    app.run(debug=True)
